# importing libraries

In [1]:
from llama_index.core.node_parser import SentenceSplitter
import numpy as np
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import faiss
from llama_index.llms.cohere import Cohere
import cohere
from cohere import Client
from llama_index.core.settings import Settings

from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate

In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

-----------------------------------------------

# query_pipeline example

# Document handling

### doc loader

In [4]:
class doc_loader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load_text(self):
        with open(self.file_path, 'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()
        return text

### doc splitter

In [5]:
class doc_splitter:
    def __init__(self, chunk_size=512, chunk_overlap= 16):
        self.splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, paragraph_separator="/n/n/n/n")

    def split(self, text):
        chunks = self.splitter.split_text(text)
        return chunks

# Embedding Handling

In [6]:
class embed_model_class:
    def __init__(self, embed_model_name):
        self.embed_model = HuggingFaceEmbedding(model_name=embed_model_name)

    def get_embedding(self, chunks):
        embedding = [self.embed_model.get_text_embedding(chunk) for chunk in chunks]
        return np.array(embedding)

# FAISS Indexing

In [7]:
class Faiss_indexing:
    def __init__(self, dimension):
        self.index = faiss.IndexFlatL2(dimension)

    def indexing(self, embeddings):
        embeddings = np.array(embeddings, dtype=np.float32)

        if embeddings.shape[1] != self.index.d:
            raise ValueError(f"Embedding dimension mismatch: Expected {self.index.d}, got {embeddings.shape[1]}")
        self.index.add(embeddings)

# Creating Query Engine

In [8]:
api_key = CO_API_KEY
cohere_client = Client(api_key)

class Query_engine:
    def __init__(self, llm_model="command-r-plus", apikey=CO_API_KEY):
        # Initialize the Cohere client with the provided API key
        self.llm = cohere.Client(apikey)
        self.model = llm_model

    def query(self, search_content, query):
        """
        Takes the searched similar content (text chunk) and the original query,
        and processes them using the LLM to generate a coherent response.
        """
        context = "\n".join(search_content)  # Join search content into a single string for context

        try:
            # Query the LLM model with the provided context and query
            response = self.llm.generate(
                model=self.model,  # Use the provided model
                prompt=f"Query: {query}\nContext: {context}",
                max_tokens=150,
                temperature=0.7
            )
            
            # Return the LLM response text
            return response.generations[0].text  # Adjust this to match the response structure of the `cohere` client

        except Exception as e:
            # If an error occurs, print it and return a friendly message
            print(f"Error querying LLM: {e}")
            return "Sorry, there was an error processing your request."


# Vector Store Abstraction

In [9]:
class FaissVectorStore:
    def __init__(self, file_path, embed_model, chunk_size=512, chunk_overlap=16):
        # Instead of creating new instances of doc_loader, doc_splitter, embed_model_class inside the class,
        # we pass the instances to the constructor.
        self.document_loader = doc_loader(file_path)
        self.document_splitter = doc_splitter(chunk_size, chunk_overlap)
        self.embedding_model = embed_model
        self.index = None

    def build_index(self):
        # Use the passed instances directly
        text = self.document_loader.load_text()
        self.chunks = self.document_splitter.split(text)

        # Generate embeddings for the chunks using the passed embed_model instance
        embeddings = self.embedding_model.get_embedding(self.chunks)

        # Create a FAISS index with the appropriate dimension (based on embedding size)
        dimension = embeddings.shape[1]
        self.index = Faiss_indexing(dimension)
        
        # Add the embeddings to the FAISS index
        self.index.indexing(embeddings)

    def search(self, query, k=5):
        # Convert the query into an embedding using the passed embed_model instance
        query_embedding = self.embedding_model.get_embedding([query])

        # Perform a similarity search using the FAISS index
        distances, indices = self.index.index.search(query_embedding, k)
        return distances, indices


    def get_text_by_index(self, idx):
        """Retrieve the text chunk based on the index."""
        # Ensure the index is valid and return the corresponding chunk
        if idx < len(self.chunks):
            return self.chunks[idx]
        else:
            raise IndexError("Index out of range")

In [10]:
# Initialize the components outside of FaissVectorStore
file_path = "data/pg10763.txt"
embed_model = embed_model_class("BAAI/bge-small-en-v1.5")

# Pass the created instances to the FaissVectorStore constructor
vector_store = FaissVectorStore(file_path=file_path, embed_model=embed_model)

# Build the index (this will load, split, embed, and index the document)
vector_store.build_index()

# Example query for searching
query = "What do the Sikh Stoics believe?"

# Perform similarity search for the query
distances, indices = vector_store.search(query, k=5)

# Print the results
print(f"Distances: {distances}")
print(f"Indices: {indices}")


index_to_retrieve = 147
chunk_text = vector_store.get_text_by_index(index_to_retrieve)

# Print the chunk text
print(f"\n\nText for index {index_to_retrieve}: {chunk_text}")

Distances: [[0.8233491  0.85043573 0.8664963  0.8756935  0.87710845]]
Indices: [[147 144  64 152 151]]


Text for index 147: _John Keats._




PRAISE THE GENEROUS GODS FOR GIVING


Some of us find joy in toil, some in art, some in the open air and the
sunshine. All of us find it in simply being alive. Life is the gift no
creature in his right mind would part with. As Milton asks,

  "For who would lose,
  Though full of pain, this intellectual being,
  These thoughts that wander through eternity,
  To perish rather, swallowed up and lost
  In the wide womb of uncreated night,
  Devoid of sense and motion?"


  Praise the generous gods for giving
    In a world of wrath and strife,
  With a little time for living,
    Unto all the joy of life.

  At whatever source we drink it,
    Art or love or faith or wine,
  In whatever terms we think it,
    It is common and divine.

  Praise the high gods, for in giving
    This to man, and this alone,
  They have made his chance of living
    Sh

# Query_pipeline

In [28]:
indices_flatterend = indices.flatten() if len(indices.shape) >1 else indices

context = []

for idx in  indices_flatterend:
    try:
        chunk_text = vector_store.get_text_by_index(idx)
        context.append(chunk_text)
    except IndexError:
        print(f"Error: Index {idx} out of range.")

In [29]:
def cohere_generate(prompt: str, max_tokens: int = 150, model: str = 'command-r-plus'):
    """Generate a response from Cohere's language model"""
    try:
        response = cohere_client.generate(
            model=model,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response.text.strip()  # Extract the generated text
    except Exception as e:
        print(f"Error generating response with Cohere: {e}")
        return "Sorry, something went wrong with the generation."

In [31]:
# from llama_index.core import BaseRetriever
import numpy as np

class FaissRetriever():
    def __init__(self, faiss_vector_store, k=5):
        self.faiss_vector_store = faiss_vector_store
        self.k = k  # Number of top results to retrieve
    
    def retrieve(self, query: str):
        # Get the embeddings for the query
        distances, indices = self.faiss_vector_store.search(query, k=self.k)
        
        # Retrieve the corresponding text chunks based on indices
        context = []
        indices_flat = indices.flatten() if len(indices.shape) > 1 else indices
        
        for idx in indices_flat:
            try:
                chunk_text = self.faiss_vector_store.get_text_by_index(idx)
                context.append(chunk_text)
            except IndexError:
                print(f"Error: Index {idx} out of range.")
        
        return context


In [39]:
from cohere import Client
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate
from llama_index.core.settings import Settings

# Initialize the Cohere client with the provided API key
class Query_engine:
    def __init__(self, llm_model="command-r-plus", apikey="CO_API_KEY"):
        self.llm = Client(apikey)
        self.model = llm_model

    def generate(self, prompt: str, max_tokens: int = 150):
        response = self.llm.generate(
            model=self.model,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response.text.strip()

# Initialize your custom Query_engine (Cohere client)
cohere_query_engine = Query_engine(apikey="CO_API_KEY")

# Create the prompt templates
prompt_str1 = "Retrieve context about the following topic: {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)

prompt_str2 = """Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

{query_str}

Syntesized response:
"""
prompt_tmpl2 = PromptTemplate(prompt_str2)

# Custom FAISS Retriever (as defined before)
class FaissRetriever():
    def __init__(self, faiss_vector_store, k=5):
        self.faiss_vector_store = faiss_vector_store
        self.k = k
    
    def retrieve(self, query: str):
        distances, indices = self.faiss_vector_store.search(query, k=self.k)
        
        context = []
        indices_flat = indices.flatten() if len(indices.shape) > 1 else indices
        
        for idx in indices_flat:
            try:
                chunk_text = self.faiss_vector_store.get_text_by_index(idx)
                context.append(chunk_text)
            except IndexError:
                print(f"Error: Index {idx} out of range.")
        
        return context

# Initialize the FAISS Retriever
faiss_retriever = FaissRetriever(faiss_vector_store=vector_store, k=5)

# QueryPipeline using Cohere
p = QueryPipeline(
    chain=[
        prompt_tmpl1,
        faiss_retriever,  # Use FAISS-based retriever here
        prompt_tmpl2,
        cohere_query_engine.generate,  # Use the Cohere LLM generation
    ],
    verbose=True
)

# Example query
query = "What do the Sikh Stoics believe?"

# Running the query
response = p.query(query)

print(response)


ValueError: Chain must be a sequence of modules or module keys.

In [41]:
from cohere import Client
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate
from llama_index.core.settings import Settings

# Initialize the Cohere client with the provided API key
class Query_engine:
    def __init__(self, llm_model="command-r-plus", apikey="CO_API_KEY"):
        self.llm = Client(apikey)
        self.model = llm_model

    def generate(self, prompt: str, max_tokens: int = 150):
        response = self.llm.generate(
            model=self.model,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response.text.strip()

# Custom Cohere Generator class for the QueryPipeline
class CohereGenerator:
    def __init__(self, query_engine: Query_engine):
        self.query_engine = query_engine

    def __call__(self, context: str) -> str:
        # Use the Cohere query engine to generate a response from the context
        prompt = f"Context: {context}\nSynthesize the answer using modern slang."
        return self.query_engine.generate(prompt)

# Initialize your custom Query_engine (Cohere client)
cohere_query_engine = Query_engine(apikey="CO_API_KEY")
cohere_generator = CohereGenerator(query_engine=cohere_query_engine)

# Create the prompt templates
prompt_str1 = "Retrieve context about the following topic: {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)

prompt_str2 = """Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

{query_str}

Syntesized response:
"""
prompt_tmpl2 = PromptTemplate(prompt_str2)

# Custom FAISS Retriever (as defined before)
class FaissRetriever():
    def __init__(self, faiss_vector_store, k=5):
        self.faiss_vector_store = faiss_vector_store
        self.k = k
    
    def retrieve(self, query: str):
        distances, indices = self.faiss_vector_store.search(query, k=self.k)
        
        context = []
        indices_flat = indices.flatten() if len(indices.shape) > 1 else indices
        
        for idx in indices_flat:
            try:
                chunk_text = self.faiss_vector_store.get_text_by_index(idx)
                context.append(chunk_text)
            except IndexError:
                print(f"Error: Index {idx} out of range.")
        
        return context

# Initialize the FAISS Retriever
faiss_retriever = FaissRetriever(faiss_vector_store=vector_store, k=5)

# QueryPipeline using Cohere
p = QueryPipeline(
    chain=[
        prompt_tmpl1,
        faiss_retriever,  # Use FAISS-based retriever here
        prompt_tmpl2,
        cohere_generator,  # Use the custom Cohere generator here
    ],
    verbose=True
)

# Example query
query = "What do the Sikh Stoics believe?"

# Running the query
response = p.query(query)

print(response)


ValueError: Chain must be a sequence of modules or module keys.

In [44]:
class FaissRetriever():
    def __init__(self, faiss_vector_store, k=5):
        self.faiss_vector_store = faiss_vector_store
        self.k = k
    
    def retrieve(self, query: str):
        distances, indices = self.faiss_vector_store.search(query, k=self.k)
        
        context = []
        indices_flat = indices.flatten() if len(indices.shape) > 1 else indices
        
        for idx in indices_flat:
            try:
                chunk_text = self.faiss_vector_store.get_text_by_index(idx)
                context.append(chunk_text)
            except IndexError:
                print(f"Error: Index {idx} out of range.")
        
        return context

In [42]:
from cohere import Client
from llama_index.core.query_pipeline import QueryPipeline
from llama_index.core import PromptTemplate
from llama_index.core.settings import Settings

# Initialize the Cohere client with the provided API key
class Query_engine:
    def __init__(self, llm_model="command-r-plus", apikey="CO_API_KEY"):
        self.llm = Client(apikey)
        self.model = llm_model

    def generate(self, prompt: str, max_tokens: int = 150):
        response = self.llm.generate(
            model=self.model,
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=0.7
        )
        return response.text.strip()

# Custom Cohere Generator class for the QueryPipeline
class CohereGenerator:
    def __init__(self, query_engine: Query_engine):
        self.query_engine = query_engine

    def __call__(self, context: str) -> str:
        # Use the Cohere query engine to generate a response from the context
        prompt = f"Context: {context}\nSynthesize the answer using modern slang."
        return self.query_engine.generate(prompt)

# Initialize your custom Query_engine (Cohere client)
cohere_query_engine = Query_engine(apikey="CO_API_KEY")
cohere_generator = CohereGenerator(query_engine=cohere_query_engine)

# Create the prompt templates
prompt_str1 = "Retrieve context about the following topic: {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)

prompt_str2 = """Syntesize the context provided into an answer using modern slang, while still quoting the sources.

Context:

{query_str}

Syntesized response:
"""
prompt_tmpl2 = PromptTemplate(prompt_str2)

# Custom FAISS Retriever (as defined before)
class FaissRetriever():
    def __init__(self, faiss_vector_store, k=5):
        self.faiss_vector_store = faiss_vector_store
        self.k = k
    
    def retrieve(self, query: str):
        distances, indices = self.faiss_vector_store.search(query, k=self.k)
        
        context = []
        indices_flat = indices.flatten() if len(indices.shape) > 1 else indices
        
        for idx in indices_flat:
            try:
                chunk_text = self.faiss_vector_store.get_text_by_index(idx)
                context.append(chunk_text)
            except IndexError:
                print(f"Error: Index {idx} out of range.")
        
        return context

# Initialize the FAISS Retriever
faiss_retriever = FaissRetriever(faiss_vector_store=vector_store, k=5)

# Ensure that the `QueryPipeline` is properly initialized with callable modules
p = QueryPipeline(
    chain=[
        prompt_tmpl1,  # First step: prompt template to retrieve context
        faiss_retriever,  # Second step: FAISS retriever
        prompt_tmpl2,  # Third step: prompt template to format the response
        cohere_generator,  # Fourth step: use Cohere generator for synthesis
    ],
    verbose=True
)

# Example query
query = "What do the Sikh Stoics believe?"

# Running the query
response = p.query(query)

print(response)


ValueError: Chain must be a sequence of modules or module keys.

In [26]:
# prompt_str1 = "Retrieve context about the following topic: {topic}"
# prompt_tmpl1 = PromptTemplate(prompt_str1)

# prompt_str2 = """Syntesize the context provided into an answer using modern slang, while still quoting the sources.

# Context:

# {query_str}

# Synthesized response:
# """
# prompt_tmpl2 = PromptTemplate(prompt_str2)

In [27]:
# def cohere_response_fn(query_str: str):
#     """Use Cohere to generate a response based on a query"""
#     return cohere_generate(f"Query: {query_str}", max_tokens=150)